In [1]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import re
import string
#nltk.download("stopwords")
stopwords = stopwords.words('english')
ps = nltk.PorterStemmer()

In [2]:
data = pd.read_csv("tweets/tweets.csv")
data["tweet"].head()

c:\users\vincent\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0    WoW, #VRA! What a bounce!  https://t.co/ItnKCe...
1                                 @AltcoinDailyio $VRA
2    @CryptoDevil9 Well hard to tell, want to buy t...
3    @DrVeraSTONE @Aion Polx deve fazer uma corrida...
4    🔥 $80 billion were just poured into the market...
Name: tweet, dtype: object

In [3]:
import re
# should process numbers, emojis too
# remove anything except letters
data["tweet"]= data["tweet"].str.replace("[^a-zA-Z]", " ")
# remove words with length 2 or less
data["tweet"]= data["tweet"].apply(lambda x: " ".join ([w for w in x.split() if len (w)>2]))
# remove links
# data["tweet"]= data["tweet"].str.replace("^https?:\/\/.*[\r\n]*", " ")
#text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
data["tweet"].head()

0                 WoW VRA What bounce https ItnKCeiHCV
1                                   AltcoinDailyio VRA
2    CryptoDevil Well hard tell want buy the massiv...
3    DrVeraSTONE Aion Polx deve fazer uma corrida c...
4    billion were just poured into the markets with...
Name: tweet, dtype: object

In [4]:
# split text up into tokens
def tokenize(text):
  tokens = re.split("\W+", text)
  return tokens
data['tweet']= data['tweet'].apply (lambda x: tokenize(x.lower()))
data["tweet"].head()

0          [wow, vra, what, bounce, https, itnkceihcv]
1                                [altcoindailyio, vra]
2    [cryptodevil, well, hard, tell, want, buy, the...
3    [drverastone, aion, polx, deve, fazer, uma, co...
4    [billion, were, just, poured, into, the, marke...
Name: tweet, dtype: object

In [5]:
# remove stopwords such as a, an, the, etc
def remove_stopword(text):
  text_nostopword= [char for char in text if char not in stopwords]
  return text_nostopword
data['tweet']= data['tweet'].apply(lambda x: remove_stopword(x))
data["tweet"].head()

0                [wow, vra, bounce, https, itnkceihcv]
1                                [altcoindailyio, vra]
2    [cryptodevil, well, hard, tell, want, buy, mas...
3    [drverastone, aion, polx, deve, fazer, uma, co...
4    [billion, poured, markets, within, hour, two, ...
Name: tweet, dtype: object

In [6]:
# stem the words to finish cleaning the data
# Will use Lemmatize
def stem(tweet_no_stopword):
  text = [ps.stem ( word) for word in tweet_no_stopword]
  return text
data["tweet"]= data["tweet"].apply(lambda x: stem(x))
data["tweet"].head()

0                  [wow, vra, bounc, http, itnkceihcv]
1                                [altcoindailyio, vra]
2    [cryptodevil, well, hard, tell, want, buy, mas...
3    [drveraston, aion, polx, deve, fazer, uma, cor...
4    [billion, pour, market, within, hour, two, bit...
Name: tweet, dtype: object

In [7]:
tweets = pd.DataFrame(columns = ['id', 'date', 'time', 'content', 'replies_count', 'retweets_count', 'likes_count', 'polarity', 'sentiment'])
tweets['id'] = data['id']
tweets['date'] = data['date']
tweets['time'] = data['time']
tweets['content'] = data['tweet']
tweets['replies_count'] = data['replies_count']
tweets['retweets_count'] = data['retweets_count']
tweets['likes_count'] = data['likes_count']
tweets.head()

,id,date,time,content,replies_count,retweets_count,likes_count,polarity,sentiment
0,1407389240804139018,2021-06-22,19:25:17,"[wow, vra, bounc, http, itnkceihcv]",0,0,0,NaN,NaN
1,1407388881557803012,2021-06-22,19:23:52,"[altcoindailyio, vra]",0,0,0,NaN,NaN
2,1407388649147310091,2021-06-22,19:22:56,"[cryptodevil, well, hard, tell, want, buy, mas...",0,0,0,NaN,NaN
3,1407388556205727747,2021-06-22,19:22:34,"[drveraston, aion, polx, deve, fazer, uma, cor...",0,0,0,NaN,NaN
4,1407387983624421378,2021-06-22,19:20:17,"[billion, pour, market, within, hour, two, bit...",2,2,10,NaN,NaN


In [8]:
from datetime import date, timedelta, datetime

sent_dict = []

# get selected dates
sdate = input("Enter date from (yyyy-mm-dd): ")   # start date
sdate = datetime.strptime(sdate, '%Y-%m-%d')
edate = input("Enter date until (yyyy-mm-dd): ") # end date
edate = datetime.strptime(edate, '%Y-%m-%d')

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    day = day.strftime('%Y-%m-%d')
    sent_dict.append({'date':day, 'term_frequency':sum(tweets['date']==day)})

sentiment_data = pd.DataFrame(data=sent_dict, columns = ['date', 'term_frequency', 'sum_sentiment', 'mean_sentiment', 'weighted_mean_sentiment'])
sentiment_data['date'] = pd.to_datetime(sentiment_data['date'])  
sentiment_data.head()

Enter date from (yyyy-mm-dd):  2021-01-01
Enter date until (yyyy-mm-dd):  2021-06-21


,date,term_frequency,sum_sentiment,mean_sentiment,weighted_mean_sentiment
0,2021-01-01,82,NaN,NaN,NaN
1,2021-01-02,92,NaN,NaN,NaN
2,2021-01-03,122,NaN,NaN,NaN
3,2021-01-04,98,NaN,NaN,NaN
4,2021-01-05,104,NaN,NaN,NaN
